In [20]:
from rank_bm25 import BM25Okapi
import json
import os

tabfact_file_path = '/data1/fch123/OTT-QA/table_crawling/data/tabfact_process'
corpus = []

it = 0
for file in os.listdir(tabfact_file_path):
    file_path = os.path.join(tabfact_file_path, file)
    with open(file_path, 'r') as f:
        table = json.load(f)
        corpus.append(table['table_text_horizontal'])
    it += 1
    if (it > 100000):
        break

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [21]:
import jsonlines
from tqdm import tqdm
import numpy as np

query = []
it = 0
with jsonlines.open('simplified-nq-train.jsonl', 'r') as f:
    for i in tqdm(f):
        query.append(i['question_text'])
        it += 1
        if(it > 1000):
            break

tokenized_query = [doc.split(" ") for doc in query]

print('start')
for i in tqdm(tokenized_query):

    doc_scores = bm25.get_scores(i)
    u = np.argmax(doc_scores)
    #print(u)


1000it [00:00, 2671.00it/s]


start


  3%|▎         | 27/1001 [00:10<06:28,  2.51it/s]


KeyboardInterrupt: 

In [13]:
print(tokenized_query[0])
bm25.get_top_n(tokenized_query[0], corpus, n=1)


['which', 'is', 'the', 'most', 'common', 'use', 'of', 'opt-in', 'e-mail', 'marketing']


["row 1 is : Word is pacifier ; British English meanings is  ; Meanings common to British and American English is something or somebody that brings peace ; American English meanings is rubber teat for babies ( UK : dummy ) . row 2 is : Word is paddle ; British English meanings is a walk through shallow water , especially at the seaside ( US approx . wade , also UK usage ) ; Meanings common to British and American English is an oar used to propel a canoe , kayak or a small boat the action of using such an oar ; American English meanings is to strike a child with a paddle as a form of corporal punishment ( dated usage ) . row 3 is : Word is panda ; British English meanings is ( panda car ) police car ( slang ) ( US : zebra , black - and - white ) ; Meanings common to British and American English is Type of animal , e.g . giant panda , red panda ; American English meanings is  . row 4 is : Word is pantomime ; British English meanings is A form of comedic , usually family oriented musical 

In [2]:
import os
from tqdm import tqdm
import gzip
import json

nq_train_data_path = '/data1/fch123/OTT-QA/data/v1.0/train/'
for it, file in enumerate(os.listdir(nq_train_data_path)):
    file_path = os.path.join(nq_train_data_path, file)
    it_num = int(file.split('-')[-1].split('.')[0])
    query = []
    query_url = []
    num_num = 0
    with gzip.open(file_path, 'r') as f:
        for i in tqdm(f):
            table = json.loads(i)
            for k, v in table.items():
                print(k)
            break
    break

0it [00:00, ?it/s]

annotations
document_html
document_title
document_tokens
document_url
example_id
long_answer_candidates
question_text
question_tokens


In [7]:
import collections
class Cell:
    def __init__(self):
        self.value_tokens: list[str] = []
        self.type: str = ""
        self.nested_tables: list[Table] = []

    def __str__(self):
        return " ".join(self.value_tokens)

    def to_dpr_json(self, cell_idx: int):
        r = {"col": cell_idx}
        r["value"] = str(self)
        return r


class Row:
    def __init__(self):
        self.cells: list[Cell] = []

    def __str__(self):
        return "| ".join([str(c) for c in self.cells])

    def visit(self, tokens_function, row_idx: int):
        for i, c in enumerate(self.cells):
            if c.value_tokens:
                tokens_function(c.value_tokens, row_idx, i)

    def to_dpr_json(self, row_idx: int):
        r = {"row": row_idx}
        r["columns"] = [c.to_dpr_json(i) for i, c in enumerate(self.cells)]
        return r


class Table(object):
    def __init__(self, caption=""):
        self.caption = caption
        self.body: list[Row] = []
        self.key = None
        self.gold_match = False

    def __str__(self):
        table_str = "<T>: {}\n".format(self.caption)
        table_str += " rows:\n"
        for i, r in enumerate(self.body):
            table_str += " row #{}: {}\n".format(i, str(r))

        return table_str

    def get_key(self) -> str:
        if not self.key:
            self.key = str(self)
        return self.key

    def visit(self, tokens_function, include_caption: bool = False) -> bool:
        if include_caption:
            tokens_function(self.caption, -1, -1)
        for i, r in enumerate(self.body):
            r.visit(tokens_function, i)

    def to_dpr_json(self):
        r = {
            "caption": self.caption,
            "rows": [r.to_dpr_json(i) for i, r in enumerate(self.body)],
        }
        if self.gold_match:
            r["gold_match"] = 1
        return r


class NQTableParser(object):
    def __init__(self, tokens, is_html_mask, title):
        self.tokens = tokens
        self.is_html_mask = is_html_mask
        self.max_idx = len(self.tokens)
        self.all_tables = []

        self.current_table = Table() # = None
        self.tables_stack = collections.deque()
        self.title = title

    def parse(self):
        self.all_tables = []
        self.tables_stack = collections.deque()

        for i in range(self.max_idx):

            t = self.tokens[i]

            if not self.is_html_mask[i]:
                # cell content
                self._on_content(t)
                continue

            if "<Table" in t:
                self._on_table_start()
            elif t == "</Table>":
                self._on_table_end()
            elif "<Tr" in t:
                self._onRowStart()
            elif t == "</Tr>":
                self._onRowEnd()
            elif "<Td" in t or "<Th" in t:
                self._onCellStart()
            elif t in ["</Td>", "</Th>"]:
                self._on_cell_end()

        return self.all_tables

    def _on_table_start(self):
        caption = self.title
        parent_table = self.current_table
        if parent_table:
            self.tables_stack.append(parent_table)

            caption = parent_table.caption
            if parent_table.body and parent_table.body[-1].cells:
                current_cell = self.current_table.body[-1].cells[-1]
                caption += " | " + " ".join(current_cell.value_tokens)

        t = Table()
        t.caption = caption
        self.current_table = t
        self.all_tables.append(t)

    def _on_table_end(self):
        t = self.current_table
        if t:
            if self.tables_stack:  # t is a nested table
                self.current_table = self.tables_stack.pop()
                if self.current_table.body:
                    current_cell = self.current_table.body[-1].cells[-1]
                    current_cell.nested_tables.append(t)
        else:
            pass

    def _onRowStart(self):
        self.current_table.body.append(Row())

    def _onRowEnd(self):
        pass

    def _onCellStart(self):
        current_row = self.current_table.body[-1]
        current_row.cells.append(Cell())

    def _on_cell_end(self):
        pass

    def _on_content(self, token):
        if self.current_table.body:
            current_row = self.current_table.body[-1]
            current_cell = current_row.cells[-1]
            current_cell.value_tokens.append(token)
        else:  # tokens outside of row/cells. Just append to the table caption.
            self.current_table.caption += " " + token

In [12]:
import unicodedata
import spacy as spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger", "ner", "entity_ruler"])


def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc]
def normalize(text):
    """Resolve different type of unicode encodings."""
    return unicodedata.normalize("NFD", text)


def prepare_answers(answers):
    r = []
    for single_answer in answers:
        single_answer = normalize(single_answer)
        single_answer = single_answer.lower().split(" ")  # tokenize(single_answer)
        r.append(single_answer)
    return r




In [21]:

an = [['<Table>', 'The', 'Walking', 'Dead', '(', 'season', '8', ')', ':', 'Critical', 'reception', 'by', 'episode', '<Tr>', '<Td>', '<Ul>', '<Li>', 'Season', '8', '(', '2017', '--', '18', ')', ':', 'Percentage', 'of', 'positive', 'reviews', 'tracked', 'by', 'the', 'website', 'Rotten', 'Tomatoes', '</Li>', '</Ul>', '</Td>', '</Tr>', '</Table>'], [True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True]]
#print(prepare_answers([' '.join(an[0])]))
html_mask = an[1]

In [25]:
from bs4 import BeautifulSoup
from table_utils.utils import *

soup = BeautifulSoup(' '.join(an[0]), 'html.parser')
tmp = soup.find_all('table')
rs = []
for _ in tmp:
    rs.append(_)

for it, r in enumerate(rs):
        heads = []
        rows = []
        replicate = {}
        for i, t_row in enumerate(r.find_all('tr')):
            if i == 0:
                for h in t_row.find_all(['th', 'td']):
                    h = remove_ref(h)
                    if len(h.find_all('a')) > 0:
                        tmp = (h.get_text(separator=" ").strip(), process_link(h))
                    else:
                        tmp = (h.get_text(separator=" ").strip(), [])
                    assert len(tmp) and isinstance(tmp[0], str) and isinstance(tmp[1], list)
                    heads.append(tmp)
            else:
                row = []
                for h in t_row.find_all(['th', 'td']):
                    col_idx = len(row)
                    if col_idx in replicate:
                        row.append(replicate[col_idx][1])
                        replicate[col_idx][0] = replicate[col_idx][0] - 1
                        if replicate[col_idx][0] == 0:
                            del replicate[col_idx]
                    h = remove_ref(h)
                    if len(h.find_all('a')) > 0:
                        tmp = (h.get_text(separator=" ").strip(), process_link(h))
                    else:
                        tmp = (h.get_text(separator=" ").strip(), [])
                    if 'rowspan' in h.attrs:
                        # Identify row span cases
                        try:
                            num = int(h['rowspan'])
                            replicate[len(row)] = [num - 1, tmp]
                        except Exception:
                            pass
                    assert len(tmp) and isinstance(tmp[0], str) and isinstance(tmp[1], list)
                    row.append(tmp)
    
                    rows.append(row)
        rows = rows[:20]
        print(heads)
        print(rows)

[('Season 8 ( 2017 -- 18 ) : Percentage of positive reviews tracked by the website Rotten Tomatoes', [])]
[]
